In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
from bokeh.plotting import figure, show, Figure
from bokeh.models import ColumnDataSource, Label
from bokeh.models.glyphs import Text
from bokeh.palettes import Spectral3
from bokeh.layouts import row, column, gridplot

# tell bokeh to show the figures in the notebook
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
dataFrame = pd.read_csv('./data/DWH_Training.csv')
dataFrameTest = pd.read_csv('./data/DWH_test.csv')

In [4]:
dataFrame.head()

,index,height(cm),weight(kg),gender
0,1,173.685185,69.158060,-1
1,2,168.295454,55.384885,1
2,3,170.213057,63.701701,1
3,4,168.673799,52.104581,1
4,5,174.530740,70.698139,-1


In [5]:
dataFrameTest.head()

,index,height(cm),weight(kg),gender,misc
0,1,174.731425,68.346183,-1,4.055077
1,2,167.961304,61.551896,1,-3.245166
2,3,167.291562,69.278931,1,-3.284287
3,4,169.423971,58.349019,1,-2.047819
4,5,175.868380,69.440505,-1,5.277262


In [6]:
dataFrame['height(cm)'].min()
dataFrame['weight(kg)'].min()

42.5408770591469

In [7]:
palette = ["SpringGreen", "Crimson"]
dataFrame['color'] = np.where(dataFrame['gender']==1,"SpringGreen", "Crimson")
dataFrame['legend_values'] = np.where(dataFrame['gender']==1,"Male", "Female")


#aff a figure
p=figure(x_axis_label='height(cm)',y_axis_label='weight(kg)', width=800, height=500,
         title='Relation between Male and Female',
        )

#render the graph
p.circle('height(cm)','weight(kg)',source=dataFrame,size=5,alpha=0.8,color='color',legend='legend_values')
p.line(x=171, y=[40,85], line_width=2, color="red")
p.line(x=[163,180], y=67, line_width=2, color="blue")
#annotate the graphic
p.y_range.start = dataFrame['weight(kg)'].min()
p.x_range.start = dataFrame['height(cm)'].min()
show(p)

In [8]:
def scatter( self, source, x, y, **kwargs ):
    # access the figure using the self variable
    self.circle( source=source, x=x, y=y, **kwargs)
    
    label = Label( x=50, y=50, x_units='screen', y_units='screen', render_mode='css' )
    self.add_layout(label)

# add the function as class method to Figure    
Figure.scatter = scatter

In [9]:
w = [0.576, 0.047]
b = [-103,-102,-101,-100,-99]
hyperPlaneValues = []
for index, row in dataFrame.iterrows():
    x = row['height(cm)']
    elements = []
    for bias in b:
        elements.append((-(w[0]/w[1])*x)-(bias/w[1]))
    hyperPlaneValues.append(elements)

normalizedList = []
heightWeightMap1 = {'height':[],
                   'weight':[]
                  }
heightWeightMap2 = {'height':[],
                   'weight':[]
                  }
heightWeightMap3 = {'height':[],
                   'weight':[]
                  }
heightWeightMap4 = {'height':[],
                   'weight':[]
                  }
heightWeightMap5 = {'height':[],
                   'weight':[]
                  }
for row in hyperPlaneValues:
    normalizedElement = []
    
    for term in row:
        if term < 0.0:
            term = 0.0
        normalizedElement.append(term)
    normalizedList.append(normalizedElement)

i = 0
for index, row in dataFrame.iterrows():
    x = row['height(cm)']
    
    if normalizedList[i][0]!=0:
        heightWeightMap1['height'].append(x)
        heightWeightMap1['weight'].append(normalizedList[i][0])
    
    if normalizedList[i][1]!=0:
        heightWeightMap2['height'].append(x)
        heightWeightMap2['weight'].append(normalizedList[i][1])
    
    if normalizedList[i][2]!=0:
        heightWeightMap3['height'].append(x)
        heightWeightMap3['weight'].append(normalizedList[i][2])
    
    if normalizedList[i][3]!=0:
        heightWeightMap4['height'].append(x)
        heightWeightMap4['weight'].append(normalizedList[i][3])
    
    if normalizedList[i][4]!=0:
        heightWeightMap5['height'].append(x)
        heightWeightMap5['weight'].append(normalizedList[i][4])
    
    i+=1
    

In [10]:
palette = ["SpringGreen", "Crimson"]
dataFrame['color'] = np.where(dataFrame['gender']==1,"SpringGreen", "Crimson")
dataFrame['legend_values'] = np.where(dataFrame['gender']==1,"Male", "Female")


#aff a figure
p=figure(x_axis_label='height(cm)',y_axis_label='weight(kg)', width=800, height=500,
         title='Relation between Male and Female',
        )

#render the graph
p.circle('height(cm)','weight(kg)',source=dataFrame,size=5,alpha=0.8,color='color',legend='legend_values')
p.line(x= heightWeightMap1['height'], y=heightWeightMap1['weight'], line_width=2, color="red")
p.line(x= heightWeightMap2['height'], y=heightWeightMap2['weight'], line_width=1, color="blue")
p.line(x= heightWeightMap3['height'], y=heightWeightMap3['weight'], line_width=2, color="green")
p.line(x= heightWeightMap4['height'], y=heightWeightMap4['weight'], line_width=2, color="yellow")
p.line(x= heightWeightMap5['height'], y=heightWeightMap5['weight'], line_width=2, color="black")
#annotate the graphic
p.y_range.start = dataFrame['weight(kg)'].min()
p.y_range.end = dataFrame['weight(kg)'].max()
p.x_range.start = dataFrame['height(cm)'].min()
p.x_range.end = dataFrame['height(cm)'].max()
show(p)

In [11]:
w = np.array([[0.576,0.047]
              ])
normOfW = math.sqrt(math.pow(0.576,2)+math.pow(0.047,2))

trainingDataPlane = []

for index, row in dataFrame.iterrows():
    
    biasList = []
    x = np.array([[row['height(cm)']],
                   [row['weight(kg)']]
                  ])
    for bias in b:
        biasList.append((np.dot(w,x)+bias)/normOfW)
    
    trainingDataPlane.append(biasList)
i = 0
sumList = [0,0,0,0,0]

for index, row in dataFrame.iterrows():
    l = 0
    gender = row['gender']
    for bias in b:
        if (trainingDataPlane[i][l][0][0] > 0.0 and gender < 0.0) or (trainingDataPlane[i][l][0][0] < 0.0 and gender > 0.0):
            sumList[l]+=1
        l+=1
    i+=1
print(sumList)

[186, 217, 205, 182, 151]


In [12]:
for index, row in dataFrameTest.iterrows():
    biasList = []
    x = np.array([[row['height(cm)']],
                   [row['weight(kg)']]
                  ])
    for bias in b:
        biasList.append((np.dot(w,x)+bias)/normOfW)
    
    trainingDataPlane.append(biasList)
i = 0
sumListTest = [0,0,0,0,0]

for index, row in dataFrameTest.iterrows():
    l = 0
    gender = row['gender']
    for bias in b:
        if (trainingDataPlane[i][l][0][0] > 0.0 and gender < 0.0) or (trainingDataPlane[i][l][0][0] < 0.0 and gender > 0.0):
            sumListTest[l]+=1
        l+=1
    i+=1
print(sumListTest)

[25, 26, 25, 20, 17]
